<a href="https://colab.research.google.com/github/NVHien10/BD_DQ/blob/main/BD_DotQuy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']

def main():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'google_auth_oauthlib'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BIGDATA/data_dotquy.csv')
df

LÀM SẠCH và CHUẨN HÓA DL
- 1 giới tính: loại other
- 2 tuổi: Loại số thực ( vd: 0.1 , 1.56) rồi dùng kỹ thuật Age Grouping
- 3 Đã kết hôn: chuyển về 0 và một
- 4 kiểu làm việc(work_type)
    + kỹ thuật Hashing Encoding
    + kỹ thuật One-Hot Encoding
- 5 vùng sống: chuyển về 0 và một
- 6 BMI  
    + Loại bỏ các hàng hoặc cột có N/A rồi dùng kỹ thuật Binary Encoding
    + dùng kỹ thuật Simple Imputation rồi dùng kỹ thuật tiếp Min-Max Scaling
- 7 smoking_status
  + Loại bỏ các hàng hoặc cột có Unknown rồi dùng kỹ thuật tiếp Min-Max Scaling
  + kỹ thuật Frequency Encoding
- 8 avg_glucose_level (mg/dL)
    DÙNG KỸ THUẬT: Min-Max Scaling
  ******ĐIỀU KIỆN LÀ******
    Người bình thường:
     < 140
    Người Bị tiểu đường:
      >=140
      

In [ ]:
#Kiem tra trung lap
duplicated_rows = df[df.duplicated(subset='id')]
print("So luong du lieu trung lap la:" , len(duplicated_rows))

df = df.dropna()
df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )

In [ ]:
# Xóa cột mã định danh
df  = df.drop(columns=['id'])
df.info()
df

In [ ]:

# khong tieu
def glucose_normal(value):
    if value < 140:
        return value
    else:
        return 0
normal_data = [glucose_normal(x) for x in df['avg_glucose_level']]
#print(normal_data)

# Calculate min and max values
X_min = min(normal_data)
X_max = max(normal_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in normal_data]
scaled_data
df['glucose_normal'] = scaled_data

In [ ]:

# tieu duong
def glucose_classify(value):
    if value >= 140:
        return value
    else:
        return 0

classified_data = [glucose_classify(x) for x in df['avg_glucose_level']]
#classified_data

# Calculate min and max values
X_min = min(classified_data)
X_max = max(classified_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in classified_data]
scaled_data
df['glucose_classify'] = scaled_data


In [ ]:
# chuẩn hóa bằng Min-Max Scaling
glucose_values = df['avg_glucose_level']

# Calculate min and max values
X_min = min(df['avg_glucose_level'])
X_max = max(df['avg_glucose_level'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['avg_glucose_level']]

# Print the original and scaled data
#print(scaled_data)
#df['glucose_normalized_data'] = scaled_data

df['avg_glucose_level'] = scaled_data
df


In [ ]:
# LOAI GIA TRI TUOI LÀ SO THỤC KHONG NGUYEN
df = df[~df['age'].apply(lambda x: isinstance(x, float) and not x.is_integer())]
df

In [ ]:
bins = [0, 18, 35, 60, 100]
labels = ['Trẻ em', 'Thanh niên', 'Trung niên', 'Người già']
df['nhóm tuổi'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
df

In [ ]:
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df

In [ ]:
#phan nhom
df.info()

In [ ]:
!git config --global user.email "hiennguyenwinh@gmail.com"
!git config --global user.name "NVHien10"

In [ ]:
!git remote add origin https://github.com/NVHien10/BD_DQ.git


In [ ]:
!apt-get install git -y